In [11]:
# %pip install folium
# %pip install mplleaflet
# %pip install selenium
# %pip install kaleido
# %pip install geopandas there is some issue with downloading geopandas

UsageError: Line magic function `%sudo` not found.


In [2]:
import pandas as pd
import numpy as np

## Only run once
# df = pd.read_csv('../data/acled/africa.csv')
# sudan = df[df["COUNTRY"].str.contains("Sudan")]
# sudan.to_csv('../data/acled/sudan.csv', index=False)

sudan = pd.read_csv('../data/acled/sudan.csv')
sudan.shape

(31716, 31)

In [3]:
country = sudan['COUNTRY'].value_counts()
country

COUNTRY
Sudan          22490
South Sudan     9226
Name: count, dtype: int64

In [4]:
import pandas as pd
import plotly.express as px

# Create a DataFrame with coordinates inside Sudan and South Sudan
data = pd.DataFrame({
    'Location': ['Location 1', 'Location 2', 'Location 3'],
    'Latitude': [12.6392, 15.5000, 9.1754],  # Example latitude values
    'Longitude': [29.4835, 30.0000, 7.1807]  # Example longitude values
})

# Create a scatter plot using Plotly
fig = px.scatter_geo(data, lat='Latitude', lon='Longitude', text='Location')

# Update the layout to set the map projection to 'equirectangular' (a static map projection)
fig.update_geos(projection_type="equirectangular")

# Customize the layout (title, axis labels, etc.) as needed
fig.update_layout(
    title='Locations in Sudan and South Sudan',
    geo=dict(
        showland=True,
    )
)

# Save the plot as a static image (e.g., PNG)
fig.write_image('static_map.png')

print("Static map saved as 'static_map.png'.")


Static map saved as 'static_map.png'.
